In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import robin_stocks as rs

In [ ]:
# basic login
rs.login(username=your_username,
         password=your_password,
         expiresIn=86400,
         by_sms=True)

In [ ]:
# syntax for setting environment variables in terminal on Mac/Linux

export robinhood_username="your_username_here"
export robinhood_password="your_password_here"

In [ ]:
# syntax for setting environment variables in command prompt on Windows

set robinhood_username=your_username_here
set robinhood_password=your_password_here

In [ ]:
# login with environment variables

import os 

robin_user = os.environ.get("robinhood_username")
robin_pass = os.environ.get("robinhood_password")

rs.login(username=robin_user,
         password=robin_pass,
         expiresIn=86400,
         by_sms=True)

In [ ]:
# logout

rs.logout()

In [ ]:
# market buy by dollar amount (stocks)

rs.orders.order_buy_fractional_by_price(symbol,
                                       ammountInDollars,
                                       timeInForce='gtc',
                                       extendedHours=False) 

In [ ]:
# example 

rs.orders.order_buy_fractional_by_price('AAPL', 500, timeInForce='gtc', extendedHours=False)


In [ ]:
# market buy by integer/fraction quantity of shares (stocks)

rs.orders.order_buy_fractional_by_quantity(symbol,
                                          quantity,
                                          timeInForce='gtc',
                                          extendedHours=False)

In [ ]:
# limit buy of stocks

rs.orders.order_buy_limit(symbol,
                          quantity,
                          limitPrice,
                          timeInForce='gtc',
                          extendedHours=False)

In [ ]:
# example

rs.orders.order_buy_limit('AAPL,
                          5,
                          450,
                          timeInForce='gtc',
                          extendedHours=False)

In [ ]:
# market buy crypto by dollar amount example

rs.orders.order_buy_crypto_by_price('ETH', 
                                 1000,
                                 timeInForce='gtc')

In [ ]:
# market buy crypto by fractional quantity amount example

rs.orders.order_buy_crypto_by_quantity('ETH', 
                                 15.9,
                                 timeInForce='gtc')


In [ ]:
# limit buy crypto example

rs.orders.order_buy_crypto_limit('BTC', 
                                 0.5,
                                 5000,
                                 timeInForce='gtc')

In [ ]:
# limit buy options

rs.orders.order_buy_option_limit(positionEffect, 
                                 creditOrDebit, 
                                 price, 
                                 symbol,
                                 quantity,
                                 expirationDate, 
                                 strike, 
                                 optionType='both', 
                                 timeInForce='gtc')

In [ ]:
# limit buy options example

rs.orders.order_buy_option_limit('open',
                                 'debit',
                                 350,
                                 "AAPL",
                                 1,
                                 '2020-07-17', 
                                 400,
                                 optionType='call',
                                 timeInForce='gtc')

In [ ]:
from time import sleep
import pandas as pd

In [ ]:
# place order for Visa when Mastercard hits certain price FULL EXAMPLE

while True:
    try:
        price = rs.stocks.get_latest_price('MA', includeExtendedHours=True)
        # assigning price to first (and only) item of list and converting from str to float
        mastercard_price = float(price[0])
        
        if mastercard_price < 280:
            try:
                rs.orders.order_buy_fractional_by_price('V', 500)
                break
                
            except Exception as e:
                print("Error placing order:", e)
        else:
            sleep(15)
                
    except Exception as e:
        print("Error fetching latest price:", e)
        
print("ORDER TRIGGERED at {}".format(pd.Timestamp.now()))
        

In [ ]:
np.random.randint(low, high=None)


In [ ]:
# place order for Visa when Mastercard moves more than 1% in last hour FULL EXAMPLE

df = pd.DataFrame(columns=['date', 'price'])

while True:
    try:
        price = rs.stocks.get_latest_price('MA', includeExtendedHours=True)
        # assigning price to first (and only) item of list and converting from str to float
        mastercard_price = float(price[0])
        
        df.loc[len(df)] = [pd.Timestamp.now(), mastercard_price]
        
        start_time = df.date.iloc[-1] - pd.Timedelta(minutes=60)
        df = df.loc[df.date >= start_time] # cuts dataframe to only include last hour of data
        max_price = df.price.max()
        min_price = df.price.min()
        
        if df.price.iloc[-1] < max_price * 0.99:
            try:
                rs.orders.order_buy_fractional_by_price('V', 500)
                print("DROPPED 1%, CURRENT PRICE: {} MAX PRICE: {}".format(df.price.iloc[-1], max_price))
                break
                
            except Exception as e:
                print("Error placing order:", e)
                
        elif df.price.iloc[-1] > min_price * 1.01:
            try:
                rs.orders.order_sell_fractional_by_price('V', 500)
                print("RISEN 1%, CURRENT PRICE: {} MIN PRICE: {}".format(df.price.iloc[-1], min_price))
                break
                
            except Exception as e:
                print("Error placing order:", e)
        
        else:
            print("NO ORDER, CURRENT PRICE: {} MIN PRICE: {} MAX PRICE: {}\n".format(df.price.iloc[-1], min_price, max_price))
            sleep(15)
                
    except Exception as e:
        print("Error fetching latest price:", e)
        
print("ORDER TRIGGERED at {}".format(pd.Timestamp.now()))
        
 

In [ ]:
# firing a pair trade when Dropbox and box diverge more than 3% over the previous day FULL EXAMPLE

dropbox_data = rs.stocks.get_stock_historicals("DBX", interval="day", span="week")
dropbox_historical = pd.DataFrame(dropbox_data)

box_data = rs.stocks.get_stock_historicals("BOX", interval="day", span="week")
box_historical = pd.DataFrame(box_data)

price_diff_yesterday = dropbox_historical.iloc[-1]['close_price'] - box_historical.iloc[-1]['close_price']

while True:
    try:
        dropbox_today = float(rs.stocks.get_latest_price('DBX', includeExtendedHours=True)[0])
        box_today = float(rs.stocks.get_latest_price('BOX', includeExtendedHours=True)[0])
        print("box today:", box_today)
        print("dropbox today:", dropbox_today)

        price_diff_today = dropbox_today - box_today

        if price_diff_today > 1.03 * price_diff_yesterday:
            try:
                # LONG BOX SHORT DROPBOX
                rs.orders.order_buy_fractional_by_price('BOX',
                                           500,
                                           timeInForce='gtc',
                                           extendedHours=False) 

                rs.orders.order_sell_fractional_by_price('DBX',            
                                           500,
                                           timeInForce='gtc',
                                           extendedHours=False) 

                print("Diverged MORE THAN 3%, YESTERDAY'S DIFFERENCE: {} TODAY'S DIFFERENCE: {} PERCENTAGE CHANGE: {}%\n".format(price_diff_yesterday, price_diff_today, (price_diff_today/price_diff_yesterday - 1)*100))
                break
            except Exception as e:
                print("Error placing orders:", e)
                sleep(15)


        else:
            print("STILL WAITING, YESTERDAY'S DIFFERENCE: {} TODAY'S DIFFERENCE: {} PERCENTAGE CHANGE: {}%\n".format(price_diff_yesterday, price_diff_today, ((price_diff_today/price_diff_yesterday - 1))*100))
            sleep(15)
    except Exception as e:
        print("Error fetching latest prices:", e)
        sleep(15)

print("ORDER TRIGGERED at {}".format(pd.Timestamp.now()))

 
    

In [ ]:
# buy Tesla call option when Tesla moves more than 1% in the last hour FULL EXAMPLE

df = pd.DataFrame(columns=['date', 'price'])

while True:
    try:
        price = rs.stocks.get_latest_price('TSLA', includeExtendedHours=True)
        # assigning price to first (and only) item of list and converting from str to float
        tesla_price = float(price[0])
        
        df.loc[len(df)] = [pd.Timestamp.now(), tesla_price]

        
        start_time = df.date.iloc[-1] - pd.Timedelta(minutes=60)
        df = df.loc[df.date >= start_time] # cuts dataframe to only include last hour of data
        max_price = df.price.max()
        min_price = df.price.min()
        
        if (df.price.iloc[-1] < max_price * 0.99 or df.price.iloc[-1] > min_price * 1.01):
            try:
                # finds current best bid for option contract we want to buy
                best_bid = rs.options.find_options_by_expiration_and_strike('TSLA',
                                                                 '2020-07-17',
                                                                 1580,
                                                                 optionType='call',
                                                                 info='bid_price')
                # converts output to float
                best_bid = float(best_bid[0])
                # we place our limit bid 0.1% above the current best bid
                our_bid = 1.001 * best_bid
                
                rs.orders.order_buy_option_limit('open', 
                                 'debit', 
                                 our_bid, 
                                 'TSLA',
                                 1,
                                 '2020-07-17', 
                                 1580, 
                                 optionType='call', 
                                 timeInForce='gtc')
                print("MOVED MORE THAN 1%, TESLA CURRENT PRICE: {} MIN PRICE: {} MAX PRICE: {}\n".format(df.price.iloc[-1], min_price, max_price))
                break
                
            except Exception as e:
                print("Error placing order:", e)
        
        else:
            print("NO ORDER, TESLA CURRENT PRICE: {} MIN PRICE: {} MAX PRICE: {}\n".format(df.price.iloc[-1], min_price, max_price))
            sleep(15)
                
    except Exception as e:
        print("Error fetching latest price:", e)
        
print("ORDER TRIGGERED at {}".format(pd.Timestamp.now()))
print("LIMIT BUY FOR OPTION CALL PLACED AT:", our_bid)
        
 

In [ ]:
# get options Greek Delta (or any other Greek by changing info filter) example

delta = float(rs.options.get_option_market_data('TSLA',
                                          '2020-07-17',
                                          1580,
                                          optionType='call',
                                          info='delta')[0])

In [ ]:
# Tesla covered call  FULL EXAMPLE

ask_price = float(rs.options.find_options_by_expiration_and_strike('TSLA',
                                                                   '2020-07-17',
                                                                   1600,
                                                                   optionType='call',
                                                                   info='ask_price')[0])

delta = float(rs.options.find_options_by_expiration_and_strike('TSLA',
                                                               '2020-07-17',
                                                               1600,
                                                               optionType='call',
                                                               info='delta')[0])


rs.orders.order_sell_option_limit('open',
                                  'credit',
                                  ask_price,
                                  'TSLA',
                                  1,
                                  '2020-07-17',
                                  1600,
                                  optionType='call',
                                  timeInForce='gtc')

rs.orders.order_buy_fractional_by_quantity('TSLA', 100)




In [ ]:
# how to sell a call while buying Tesla stock in a delta-hedged manner FULL EXAMPLE

ask_price = float(rs.options.find_options_by_expiration_and_strike('TSLA',
                                                                   '2020-07-17',
                                                                   1600,
                                                                   optionType='call',
                                                                   info='ask_price')[0])
rs.orders.order_sell_option_limit('open',
                                  'credit',
                                  ask_price,
                                  'TSLA',
                                  1,
                                  '2020-07-17',
                                  1600,
                                  optionType='call',
                                  timeInForce='gtc')

delta = float(rs.options.find_options_by_expiration_and_strike('TSLA',
                                                               '2020-07-17',
                                                               1600,
                                                               optionType='call',
                                                               info='delta')[0])

rs.orders.order_buy_fractional_by_quantity('TSLA', delta*100)

In [ ]:
# place a stoploss and cancel and replace with market sell if target reached Robinhood work-around FULL EXAMPLE

rs.orders.order_buy_fractional_by_quantity('AAPL', 10)

stoploss_order = rs.orders.order_sell_stop_loss('AAPL', 10, 370)
stoploss_order_id = stoploss_order['id']

while True:
    price = rs.stocks.get_latest_price('AAPL', includeExtendedHours=True)
    price = float(price[0]) # convert to single float

    if price > 400:
        rs.orders.cancel_stock_order(stoploss_order_id)
        rs.orders.order_sell_market('AAPL', 10)
        print("STOP LOSS CANCELLED AND MARKET SELL TRIGGERED, APPLE PRICE:", price)
        print("ORDER TRIGGERED at {}".format(pd.Timestamp.now()))
        break
    
    print("STILL WAITING, APPLE PRICE:", price)
    sleep(15)

In [ ]:
# get all account positions

rs.account.get_all_positions()

In [ ]:
# get all open stock orders

rs.orders.get_all_open_stock_orders()

In [ ]:
# cancel all open stock orders

rs.orders.cancel_all_stock_orders()

In [ ]:
# get Tesla historical data and convert to data frame

tesla_data= rs.stocks.get_stock_historicals("TSLA", interval="10minute", span="year")
tesla_dataframe= pd.DataFrame(tesla_data)

In [ ]:
# get options historical example

rs.options.get_option_historicals('AAPL',
                                  '2020-07-17',
                                  '350',
                                  'put',
                                  interval='5minute',
                                  span='week',
                                  bounds='regular')

In [ ]:
# get stock fundamentals

rs.stocks.get_fundamentals(inputSymbols, info=None)

In [ ]:
# get stock fundamentals for multiple tickers at once example 

rs.stocks.get_fundamentals(["AAPL,"AMZN","GOOGL"])

In [ ]:
# get Apple's dividend example

dividend = float(rs.stocks.get_fundamentals("AAPL", info='dividend_yield')[0])

In [ ]:
# get options chain for a ticker example

rs.options.get_chains("AAPL", info='expiration_dates')

In [ ]:
# find tradeable options example

rs.options.find_tradable_options('AAPL', 
                                 expirationDate=None,
                                 strikePrice=400,
                                 optionType='call')

In [ ]:
# export completed stock orders to csv example

rs.export.export_completed_stock_orders("order_history",
                                        "stocks_monday")